# Day 8 - Treetop Tree House

> The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.
>
>First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.
>
>The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:
>```
>30373
>25512
>65332
>33549
>35390
>```
>Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.
>
>A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.
>
>All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:
>
>- The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
>- The top-middle 5 is visible from the top and right.
>- The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
>- The left-middle 5 is visible, but only from the right.
>- The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
>- The right-middle 3 is visible from the right.
>- In the bottom row, the middle 5 is visible, but the 3 and 4 are not.
>
>With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Today's puzzle was interesting and I ended up going with a very straight-forward bruteforce method.

## Read input

Since we need to numerically compare numbers from different cells, I'm reading the input into a 2-dimensional array of integers.

In [63]:
from utils import read_input

def transformer(line):
    return [int(num) for num in line]

forest = read_input(8, transformer)
example = read_input(8, transformer, True)

## Part 1

> **Consider your map; how many trees are visible from outside the grid?**

Firstly, everything on the edges is visible so I'm dealing with that case right away.

Then I use way too many loops to check each tree separately to the left and right. Once that is done, I transpose the list to turn columns into rows and vice versa and repeat the process.

### Transposing matrices

I'm using a very handy Python trick here to transpose a matrix – meaning I turn the columns into rows and vice versa.

Let's look at a simpler example where we have a 3x3 grid.

```
grid = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
transposed = list(zip(*grid))
# transposed = [(1, 4, 7), (2, 5, 8), (3, 6, 9)]
```

The way it works is that [`zip`](https://docs.python.org/3/library/functions.html#zip) accepts any number of parameters and groups the first item of each together into a tuple, then the second and so on. By using `*` operator, we unpack our `grid` which in this case would be the same as writing `zip(grid[0], grid[1], grid[2])` but with the `*`, we make it general and work with any size of a matrix.

### Checking if something is visible

I use `len([tree for tree in forest[y][:x] if tree >= value]) == 0` (with varying values of `x` and `y`.

First, I find all the trees in the same row that are to the left of our value (`[:x]`) and is greater or equal to the value we want to compare to. If there are any, it means that something is blocking the view.

Second, to check for anything to the right, I change it to `[x+1:]` and do the same.

Then I transpose the matrix and do those two steps again.

In [61]:
def find_visible_count(forest):
    edges = 0, len(forest) - 1
    is_visible = []
    for y in range(edges[0], edges[1]+1):
        for x in range(edges[0], edges[1]+1):
            value = forest[y][x]
            # All the ones on the edges are visible
            if y in edges or x in edges:
                is_visible.append(value)
                continue
            from_left = len([tree for tree in forest[y][:x] if tree >= value]) == 0
            from_right = len([tree for tree in forest[y][x+1:] if tree >= value]) == 0
            if from_left or from_right:
                is_visible.append(value)
                continue

            t_forest = list(zip(*forest))
            from_top = len([tree for tree in t_forest[x][:y] if tree >= value]) == 0
            from_bottom = len([tree for tree in t_forest[x][y+1:] if tree >= value]) == 0
            if from_top or from_bottom:
                is_visible.append(value)
                continue
    return(len(is_visible))

In [64]:
solution_1 = find_visible_count(forest)
print(f'Part 1: {solution_1}')
assert solution_1 == 1840

Part 1: 1840


## Part 2

>Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.
>
>To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)
>
>The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.
>
>In the example above, consider the middle 5 in the second row:
>```
>30373
>25512
>65332
>33549
>35390
>```
>- Looking up, its view is not blocked; it can see 1 tree (of height 3).
>- Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
>- Looking right, its view is not blocked; it can see 2 trees.
>- Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).
>
>A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).
>
>However, you can do even better: consider the tree of height 5 in the middle of the fourth row:
>```
>30373
>25512
>65332
>33549
>35390
>```
>- Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
>- Looking left, its view is not blocked; it can see 2 trees.
>- Looking down, its view is also not blocked; it can see 1 tree.
>- Looking right, its view is blocked at 2 trees (by a massive tree of height 9).
>
>This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.
>
>**Consider each tree on your map. What is the highest scenic score possible for any tree?**

My solution here is similar to the first one: `scenic_score` gets a position and the forest layout and goes through all four dimensions from that point onward until it hits the edge or a value that is greater or equal to it. It then returns the product of all four direction as the score.

In `find_highest_scenic_score`, I go through every cell, find the score and return the largest one.

In [68]:
def scenic_score(pos, forest):
    y, x = pos
    value = forest[y][x]
    left, right, top, bottom = 0, 0, 0, 0
    while x != 0:
        left += 1
        if forest[y][x-1] >= value:
            break
        x -= 1
    
    y, x = pos
    while x != len(forest) - 1:
        right += 1
        if forest[y][x+1] >= value:
            break
        x += 1
        
    y, x = pos
    while y != 0:
        top += 1
        if forest[y-1][x] >= value:
            break
        y -= 1
    
    y, x = pos
    while y != len(forest) - 1:
        bottom += 1
        if forest[y+1][x] >= value:
            break
        y += 1

    return left * right * top * bottom

def find_highest_scenic_score(forest):
    highest = 0
    for y in range(0, len(forest)):
        for x in range(0, len(forest)):
            score = scenic_score((y, x), forest)
            if score > highest:
                highest = score
    return highest

In [56]:
highest_score = find_highest_scenic_score(forest)
print(f'Part 2: {highest_score}')
assert highest_score == 405769

Part 2: 405769
